# References
- Food atlas: https://www.tasteatlas.com/algeria?ref=main-menu
- Wikipedia National Dish: https://en.wikipedia.org/wiki/National_dish
- Wikipedia full dish selection : https://en.wikipedia.org/wiki/Category:Cuisine_by_country



In [ ]:
# To update ubuntu to be able to run apt install correctly
!apt-get update &> /dev/null

In [ ]:
# Install chromium browser and web driver
!apt-get install chromium-browser &> /dev/null
!apt install chromium-chromedriver &> /dev/null

In [ ]:
# Install python libraries
!pip install selenium &> /dev/null

In [ ]:
import os
import time
import uuid
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def generate_uid(row):
    # Generate UUID for each row based on 'Country' column (or any other column you choose)
    return uuid.uuid4().hex

In [ ]:
# Options to disable chromium functionalities
options = webdriver.ChromeOptions()
options.add_argument('--verbose')
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920, 1200')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options = options)

## Getting all regions and countries

In [ ]:
home_link = "https://www.tasteatlas.com/"
driver.get(home_link)
time.sleep(2)

In [ ]:
# Locating the destinations button to drill down and get the regions
destinations_button = driver.find_element(By.XPATH, '/html/body/div[2]/ta-header-desktop/header/div/ta-header-bottom/div/div/div[2]/a')
destinations_button.click()

In [ ]:
# Getting all regions to loop on
region_list = driver.find_elements(By.XPATH, '/html/body/div[2]/ta-header-desktop/header/div/ta-header-bottom/div/div[2]/div/div/div')

In [ ]:
countries_list = []
# Loop through each region to store it in the countries list later on
for region in region_list:
    # Click on the region button
    region_name = region.text
    print(region_name,"="*10,"ok")
    region.click()
    time.sleep(2)
    # Get the countries in the region
    country_list = driver.find_elements(By.XPATH, '//div[@class="submenu-item flex ng-scope"]/a[@class="submenu-item--name ng-binding"]')

    for country in country_list:
      country_url = country.get_attribute('href')
      country_name = country.text
      #print(country_name, country_url)

      countries_list.append({'Region': region_name,
                             'Country': country_name,
                             'Country_URL': country_url})

country_dataset = pd.DataFrame(countries_list)
country_dataset['UID'] = country_dataset.apply(lambda row: generate_uid(row), axis=1)

Africa ========== ok
Asia ========== ok
Australia ========== ok
Europe ========== ok
North America ========== ok
South America ========== ok


## Getting all contents for the countries

In [ ]:
country_dataset = country_dataset[:1]

In [ ]:
country_dataset

,Region,Country,Country_URL,UID
0,Africa,Algeria,https://www.tasteatlas.com/algeria?ref=main-menu,623dc4e163f046e39f3b758613458e6b


In [ ]:
#### Esta roto el loop de clickar el show more btn

In [ ]:
country_contents = []

for index, country in country_dataset.iterrows():
  country_id = country[3]
  country_link = country[2]

  print(country[1])

  driver.get(country_link)

  #containers = driver.find_elements(By.XPATH, '//div[@class="search-results__whattoeat-content ta-parent-section ng-scope"]')


  try:
    containers = WebDriverWait(driver, 4).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@class="search-results__whattoeat-content ta-parent-section ng-scope"]'))
        )
  except:
    print("Timed out waiting for containers to load")
    continue

  # Get all containers
  for container in containers:
      # Find the contents within the container (using . in the Xpath to get only the data from the container)
      container_name = container.find_elements(By.XPATH, './/h2[@class="h2 h2--lowercase h2--muli h2--large search-results__whattoeat-content-title ng-binding ng-scope"]')[0].text.strip()
      container_name = ' '.join(container_name.split(' ')[3:])


      print(container_name)

      # Clicking the "VIEW MORE" button like there is no tomorrow
      while True:
          try:
            view_more_button = WebDriverWait(container, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "btn--underscore") and contains(text(), "VIEW MORE")]'))
                )

            view_more_button.click()
            print('click show moreeeee')
          except:
            break

      # Getting the contents in the container to iterate
      contents = container.find_elements(By.XPATH, './/h2[@class="h2 h2--bold h2--lowercase"]/a[@class="ng-binding"]')

      for content in contents:
          content_text = content.text
          content_url = content.get_attribute('href')

          # Print the contents for debugging
          print(f"{container_name} - {content_text}: {content_url}")

          country_contents.append({'country id': country_id,
                                  'type': container_name,
                                  'name': content_text,
                                  'url': content_url,
                                   })

country_contents_dataset = pd.DataFrame(country_contents)
country_contents_dataset['UID'] = country_contents_dataset.apply(lambda row: generate_uid(row), axis=1)

In [ ]:
driver.get('https://www.tasteatlas.com/algeria?ref=main-menu')

try:
  containers = WebDriverWait(driver, 4).until( EC.presence_of_all_elements_located((By.XPATH, '//div[@class="search-results__whattoeat-content ta-parent-section ng-scope"]')) )
except:
  print("Timed out waiting for containers to load")

In [ ]:
for container in containers[:1]:
      # Find the contents within the container (using . in the Xpath to get only the data from the container)
      container_name = container.find_elements(By.XPATH, './/h2[@class="h2 h2--lowercase h2--muli h2--large search-results__whattoeat-content-title ng-binding ng-scope"]')[0].text.strip()
      container_name = ' '.join(container_name.split(' ')[3:])
      print(container_name)
      time.sleep(5)

      #view_more_button = container.find_elements(By.XPATH, '//button[contains(@class, "btn--underscore") and contains(text(), "VIEW MORE")]')
      #view_more_button = container.find_element(By.XPATH, './/div[contains(@class, "btn btn--underscore btn--nopadding btn--no-bgcolor btn--black-text btn--extrabold")]')
      #view_more_button = container.find_element(By.XPATH, './/div[@class="search-results__view-more ng-scope"]')
      #view_more_button = WebDriverWait(container, 10).until( EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "btn--underscore") and contains(text(), "VIEW MORE")]')) )

      iteration = 0
      while True:
        try:
          view_more_button = WebDriverWait(container, 10).until( EC.element_to_be_clickable((By.XPATH, './/div[@class="search-results__view-more ng-scope"]')) )
          iteration += 1
          view_more_button.click()
          print(f'click show moreeeee {iteration}')
          time.sleep(3)

        except:
          break

      ### Getting the contents in the container to iterate
      contents = container.find_elements(By.XPATH, './/h2[@class="h2 h2--bold h2--lowercase"]/a[@class="ng-binding"]')
      for content in contents:
          content_text = content.text
          content_url = content.get_attribute('href')

          print(f"{container_name} - {content_text}: {content_url}")

dishes
click show moreeeee 1
dishes - Couscous (dish): https://www.tasteatlas.com/Couscous-algeria
dishes - Mahjouba: https://www.tasteatlas.com/mahjouba
dishes - Harira: https://www.tasteatlas.com/harira-algeria
dishes - Chakhchoukha: https://www.tasteatlas.com/chakhchoukha
dishes - Rechta: https://www.tasteatlas.com/rechta
dishes - Chorba frik: https://www.tasteatlas.com/chorba-frik
dishes - Loubia: https://www.tasteatlas.com/loubia-algeria
dishes - Khfaf: https://www.tasteatlas.com/khfaf
dishes - Baghrir: https://www.tasteatlas.com/baghrir-algeria
dishes - Tajine zitoune: https://www.tasteatlas.com/tajine-zitoune
dishes - Msemen: https://www.tasteatlas.com/msemen-algeria
dishes - Makroud el louse: https://www.tasteatlas.com/makroud-el-louse
dishes - Garantita: https://www.tasteatlas.com/garantita
dishes - Lham lahlou: https://www.tasteatlas.com/lham-lahlou
dishes - Zalabia: https://www.tasteatlas.com/zalabia-algeria
dishes - Mkhabez: https://www.tasteatlas.com/mkhabez
dishes - Tamin